In [1]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt
from Ising_circuit_4qubits import Ising, Udis

In [25]:
h_values = np.linspace(0, 4, 100)
J = 1

In [3]:
# def build_tfim_hamiltonian(n, edges, J, h):
#     """Return the TFIM Hamiltonian H = J Σ XX + h Σ Z."""
#     coeffs = []
#     ops = []

#     # XX terms
#     for i, j in edges:
#         coeffs.append(J)
#         ops.append(qml.PauliX(i) @ qml.PauliX(j))
#         #ops.append(qml.PauliZ(i) @ qml.PauliZ(j))

#     # Z field terms
#     for i in range(n):
#         coeffs.append(h)
#         ops.append(qml.PauliZ(i))
#         #ops.append(qml.PauliX(i))

#     return qml.Hamiltonian(coeffs, ops)

# def chain_edges(n, periodic = False):
#     edges = [(i, i+1) for i in range(n-1)]
#     if periodic and n > 2:
#         edges.append((n-1, 0))
#     return edges

# def ground_state(ham): #to do : something something sparse matrix better
#     """Return (E0, |psi0>) for a PennyLane Hamiltonian."""

#     mat = qml.matrix(ham)
#     eigvals, eigvecs = np.linalg.eigh(mat)

#     idx = np.argmin(eigvals)
#     return eigvals[idx], eigvecs[:, idx]

# n = 4
# idx = 2
# h = h_values[idx]
# edges = chain_edges(n, periodic = True)
# H = build_tfim_hamiltonian(n, edges, J = J, h = h)
# H

In [17]:
def modified_ising_hamiltonian(n, J, h):
    """Ising Hamiltonian with transverse field."""
    coeffs = []
    obs = []

    # 1) nearest-neighbor XX term (open boundary)
    for i in range(n - 1):
        coeffs.append(J) 
        obs.append(qml.PauliX(i) @ qml.PauliX(i+1))

    # 2) YZZ...ZY term
    coeffs.append(J)
    op = qml.PauliY(0)
    for j in range(1, n-1):
        op = op @ qml.PauliZ(j)
    op = op @ qml.PauliY(n-1)
    obs.append(op)

    # 3) transverse field Z term
    for i in range(n):
        coeffs.append(h)
        obs.append(qml.PauliZ(i))

    H = qml.Hamiltonian(coeffs, obs)
    return H


def ground_state(ham): #to do : something something sparse matrix better
    mat = qml.matrix(ham)
    eigvals, eigvecs = np.linalg.eigh(mat)

    idx = np.argmin(eigvals)
    return eigvals[idx], eigvecs[:, idx]

n = 4
idx = 2
h = h_values[idx]
print(h)
J = 1 #use J = 1 else you need to put J into your bogoliubov circuit

H = modified_ising_hamiltonian(n = n, J = J, h = h)
print(H)

print(ground_state(H))
energy, gs = ground_state(H) 
#rho = np.outer(gs, np.conjugate(gs)) #density matrix of gs

0.08080808080808081
1 * (X(0) @ X(1)) + 1 * (X(1) @ X(2)) + 1 * (X(2) @ X(3)) + 1 * (Y(0) @ Z(1) @ Z(2) @ Y(3)) + 0.08080808080808081 * Z(0) + 0.08080808080808081 * Z(1) + 0.08080808080808081 * Z(2) + 0.08080808080808081 * Z(3)
(-4.006519320538813, array([ 0.00000000e+00+0.j, -3.39015942e-01+0.j,  3.39015942e-01+0.j,
        3.33066907e-16+0.j, -3.39015942e-01+0.j, -1.02185743e-16+0.j,
       -1.11022302e-16+0.j,  3.67516246e-01+0.j,  3.39015942e-01+0.j,
        1.11022302e-16+0.j,  1.03690634e-16+0.j, -3.67516246e-01+0.j,
       -1.11022302e-16+0.j,  3.67516246e-01+0.j, -3.67516246e-01+0.j,
       -1.11198618e-16+0.j]))


In [19]:
n_qubits = 4
dev = qml.device("default.qubit", wires = n_qubits)

@qml.qnode(dev)
def gs_circuit(h, subregion):
    #Layer of X is needed else the groundstate has the wrong sign? 
    #Understand this better https://arxiv.org/pdf/2410.21143
    for i in range(4):
        qml.PauliX(i)
    if h < 1: # prepare the initial state in |0001> else prep |0000>
        qml.PauliX(wires = 3)
    Udis(h)
    # Return the density matrix of the subregion
    #return qml.density_matrix(wires = subregion)
    H = modified_ising_hamiltonian(n = n_qubits, J = J, h = h)
    return qml.expval(H) 

E_circuit = gs_circuit(h = h, subregion = [0,1,2,3])
print(E_circuit)

-4.006519320538812
